In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2020-10-07 17:05:51--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  6.94MB/s    in 0.8s    

2020-10-07 17:05:52 (6.94 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [2]:
!unzip ml-1m.zip -d .

Archive:  ml-1m.zip
   creating: ./ml-1m/
  inflating: ./ml-1m/movies.dat      
  inflating: ./ml-1m/ratings.dat     
  inflating: ./ml-1m/README          
  inflating: ./ml-1m/users.dat       


### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [5]:
!pip install implicit > /dev/null 2>&1

In [6]:
!pip install lightfm > /dev/null 2>&1

In [7]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [8]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [9]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [10]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [11]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [12]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [13]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [16]:
!export OPENBLAS_NUM_THREADS=1

In [17]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [18]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [19]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [21]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1526    Hercules (1997)',
 '2692    Iron Giant, The (1999)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [23]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [24]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [25]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [26]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '847    Godfather, The (1972)',
 '1179    Princess Bride, The (1987)',
 '3402    Close Encounters of the Third Kind (1977)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [ ]:
!pip install -U tqdm

In [283]:
import heapq
import random
import torch
import random
from sklearn.neighbors import KDTree
from tqdm.notebook import tqdm

In [309]:
unique_users = list(set(ratings.user_id))
unique_movies = list(set(ratings.movie_id))
n_unique_users = len(unique_users)
n_unique_movies = len(unique_movies)
user_to_index = {user: index for index, user in enumerate(unique_users)}
movie_to_index = {movie: index for index, movie in enumerate(unique_movies)}

user_movie = torch.zeros(n_unique_users, n_unique_movies).float()
mask = torch.zeros_like(user_movie).bool()

iter_rows = list(zip(ratings.user_id, ratings.movie_id, ratings.rating))  # pd iterrows too slow :(
for user, movie, rating in tqdm(iter_rows, total=len(iter_rows)):
    user_index = user_to_index[user]
    movie_index = movie_to_index[movie]
    user_movie[user_index, movie_index] = r
    mask[user_index, movie_index] = 1

In [105]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
user_movie, mask = user_movie.to(device), mask.to(device)

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [304]:
class RSModelSVD:

    def __init__(self, user_movie, mask, unique_users, unique_movies, hidden_size, device):
        self.A = user_movie
        self.mask = mask
        self.movie_to_index = {movie: index for index, movie in enumerate(unique_movies)}
        self.user_to_index = {user: index for index, user in enumerate(unique_users)}
        self.unique_movies = unique_movies
        self.U = torch.empty(len(unique_users), hidden_size).to(device).uniform_(0, 1 / np.sqrt(hidden_size))
        self.V = torch.empty(hidden_size, len(unique_movies)).to(device).uniform_(0, 1 / np.sqrt(hidden_size))
        self.grad_norm_coef = torch.sum(self.mask.int())
        self.U_norm_coef = len(unique_users) * hidden_size
        self.V_norm_coef = hidden_size * len(unique_movies)
    

    def fit(self, n_max_iters=100, lr=1e2, wd=1e-2):
        """ Run fitting SVD"""

        pbar = tqdm(range(n_max_iters), total=n_max_iters, desc='Fitting SVD...')
        for _ in pbar:

            # calculate diff
            diff = self.U @ self.V - self.A
            diff[~ self.mask] = 0

            # make step
            U_grad, V_grad = diff @ self.V.T, self.U.T @ diff
            self.U -= lr * (U_grad / self.grad_norm_coef + wd * self.U / self.U_norm_coef)
            self.V -= lr * (V_grad / self.grad_norm_coef + wd * self.V / self.V_norm_coef)

            # log metric MSE
            mse = torch.sum( ((self.U @ self.V - self.A) ** 2) * self.mask.float()) / self.grad_norm_coef
            pbar.set_description(f"mse: {mse:.2f}")
        
        self.tree = KDTree(self.V.T.cpu().numpy())
    
    def recommend(self, user_id, user_item_csr, k=10):
        """ For `get_recommendations` func """

        user_index = self.user_to_index[user_id]
        ratings = (self.U @ self.V).cpu().numpy()
        user_rating = ratings[user_index]

        topk_movies_idxs = heapq.nlargest(k, enumerate(user_rating), key=lambda x: x[1])
        topk_movies_idxs = list(zip(*topk_movies_idxs))[0]
        
        result = []
        for movies_index in topk_movies_idxs:
            result.append((self.unique_movies[movies_index], movies_index))

        return result
        
    def similar_items(self, movie_id, k=10):
        """ For `get_similars` func """

        movie_index = self.movie_to_index[movie_id]
        query_points = self.V.T.cpu().numpy()[movie_index]

        distances, indexs = self.tree.query([query_points], k, return_distance=True)

        result = []
        for distance, index in zip(distances[0], indexs[0]):
            result.append((self.unique_movies[index], distance))

        return result

In [305]:
model = RSModelSVD(user_movie, mask, unique_users, unique_movies, hidden_size=64, device=device)

In [306]:
model.fit(n_max_iters=2000, lr=1e2, wd=1e-3)  # 20 sec

In [307]:
get_recommendations(4, model)

['604    Fargo (1996)',
 '911    Citizen Kane (1941)',
 '3366    Double Indemnity (1944)',
 '1120    Monty Python and the Holy Grail (1974)',
 "941    It's a Wonderful Life (1946)",
 '109    Taxi Driver (1976)',
 '900    Casablanca (1942)',
 '2327    Shakespeare in Love (1998)',
 '1189    To Kill a Mockingbird (1962)',
 "523    Schindler's List (1993)"]

In [308]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1128    Line King: Al Hirschfeld, The (1996)',
 "2286    Bug's Life, A (1998)",
 '3000    Effect of Gamma Rays on Man-in-the-Moon Marigo...',
 '958    Beat the Devil (1954)',
 '25    Othello (1995)',
 '3387    Color of Paradise, The (Rang-e Khoda) (1999)',
 '3423    Son of the Sheik, The (1926)',
 '3409    Bamba, La (1987)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных